# Compressing using DEFLATE and zlib

This notebook introduces the hardware functions for implementing DEFLATE - the compression format used by gzip and zlib. Using DEFLATE is more involved than the LZ4 algorithm introduced previously as it consists of three phases:

 1. Duplicate sections of the data are replaced by references to create an LZ77 data stream
 2. The frequency of characters and references are used to construct a huffman tree
 3. The LZ77 data stream is bit-compressed using the huffman tree

![deflate pipeline](img/deflate.png)

In the data-compression library each of these phases is represented by a different kernel. We'll walk through each stage in turn looking at how they interact. Note that each kernel follows the structure outlined in the *Introduction to Compression* notebook - specifically that they operate on 8 blocks in parallel.

First step to create our test data - for this example we are going to focus on a single  set of 8 1 MB blocks so we'll use the first 8 MB of the test data

In [1]:
with open('test_data.bin', 'rb') as f:
    test_data = f.read(8 * 1024 * 1024)

Next we can instantiate the xclbin file and get references to the kernels for each of the three stages.

In [2]:
import pynq

ol = pynq.Overlay('compression.xclbin')

lz77 = ol.xilLz77Compress_1
treegen = ol.xilTreegenKernel_1
huffman = ol.xilHuffmanKernel_1

## LZ77 Compression

Working from left to right in the above diagram we start with the LZ77 kernel and inspect the arguments to see what buffers we are going to need

In [3]:
lz77.args

{'in_r': XrtArgument(name='in_r', index=1, type='ap_uint<512> const *', mem='bank0'),
 'out_r': XrtArgument(name='out_r', index=2, type='ap_uint<512>*', mem='bank0'),
 'compressd_size': XrtArgument(name='compressd_size', index=3, type='unsigned int*', mem='bank0'),
 'in_block_size': XrtArgument(name='in_block_size', index=4, type='unsigned int*', mem='bank0'),
 'dyn_ltree_freq': XrtArgument(name='dyn_ltree_freq', index=5, type='unsigned int*', mem='bank0'),
 'dyn_dtree_freq': XrtArgument(name='dyn_dtree_freq', index=6, type='unsigned int*', mem='bank0'),
 'block_size_in_kb': XrtArgument(name='block_size_in_kb', index=7, type='unsigned int', mem=None),
 'input_size': XrtArgument(name='input_size', index=8, type='unsigned int', mem=None)}

The input and output buffers are the same as for LZ4 and we have already chosen a block size of 1 MB. Note that the LZ77 stream is 32-bit integers as each symbol can either be a literal byte or a reference consisting of a packed length and distance.

In [4]:
BLOCK_SIZE = 1024 * 1024

in_buffers = pynq.allocate((8,BLOCK_SIZE), 'u1', target=ol.bank0)
lz77_buffers = pynq.allocate((8, BLOCK_SIZE), 'u4', target=ol.bank0)

uncompressed_size = pynq.allocate((8,), 'u4', target=ol.bank0)
compressed_size = pynq.allocate((8,), 'u4', target=ol.bank0)

The buffer sizes needed for the other arguments are less obvious and are defined by how the hardware was compiled. In our case we are using the defaults so we can extract a set of constants from the libraries [configuration file](
https://github.com/Xilinx/Vitis_Libraries/blob/master/data_compression/L2/include/zlib_config.hpp).

In [5]:
LTREE_SIZE = 1024
DTREE_SIZE = 64

These are the sizes of the literal/reference size tree and the distance tree. These sizes also define the size of the frequency arrays which are output from the LZ77 compressor.

In [6]:
ltree_freq = pynq.allocate((8, LTREE_SIZE), 'u4', target=ol.bank0)
dtree_freq = pynq.allocate((8, DTREE_SIZE), 'u4', target=ol.bank0)

With all of the buffers allocated we can copy the data to the card

In [7]:
in_buffers.reshape(8*BLOCK_SIZE)[:] = memoryview(test_data)
in_buffers.sync_to_device()

uncompressed_size[:] = BLOCK_SIZE
uncompressed_size.sync_to_device()

and perform the first stage of the compression pipeline:

In [8]:
lz77.call(in_buffers, lz77_buffers,
          compressed_size, uncompressed_size,
          ltree_freq, dtree_freq,
          1024, 8*BLOCK_SIZE)

As a check to make sure that something sensible is happening we can have a look at the compressed sizes. Note that these sizes are in bytes and each symbol is now four bytes in length meaning that the number of symbols has been reduced even though their size has grown.

In [9]:
compressed_size.sync_from_device()
compressed_size

PynqBuffer([ 212596,  318440, 1440364, 1303792, 3039656, 2491536, 2452716,
            2136280], dtype=uint32)

## Generating the Huffman Tree

Next we need to take the frequency data output by the LZ77 compressor and generate the huffman tree. We can again check the arguments of the function.

In [10]:
treegen.args

{'dyn_ltree_freq': XrtArgument(name='dyn_ltree_freq', index=1, type='unsigned int*', mem='bank0'),
 'dyn_dtree_freq': XrtArgument(name='dyn_dtree_freq', index=2, type='unsigned int*', mem='bank0'),
 'dyn_bltree_freq': XrtArgument(name='dyn_bltree_freq', index=3, type='unsigned int*', mem='bank0'),
 'dyn_ltree_codes': XrtArgument(name='dyn_ltree_codes', index=4, type='unsigned int*', mem='bank0'),
 'dyn_dtree_codes': XrtArgument(name='dyn_dtree_codes', index=5, type='unsigned int*', mem='bank0'),
 'dyn_bltree_codes': XrtArgument(name='dyn_bltree_codes', index=6, type='unsigned int*', mem='bank0'),
 'dyn_ltree_blen': XrtArgument(name='dyn_ltree_blen', index=7, type='unsigned int*', mem='bank0'),
 'dyn_dtree_blen': XrtArgument(name='dyn_dtree_blen', index=8, type='unsigned int*', mem='bank0'),
 'dyn_bltree_blen': XrtArgument(name='dyn_bltree_blen', index=9, type='unsigned int*', mem='bank0'),
 'max_codes': XrtArgument(name='max_codes', index=10, type='unsigned int*', mem='bank0'),
 'block

We can see three trees with frequency, code and bit-length arrays. We've already seen the literal tree and the distance tree and the third is a tree of bit-lengths used internally by the huffman coder. Returning to the config file above we can find its size and also the size for the `max_codes` array which again contains data useful to the internal operation of the coder.

In [11]:
BLTREE_SIZE = 64
MAXCODE_SIZE = 16

We now have the information to allocate all of the arrays used by treegen kernel

In [12]:
bltree_freq = pynq.allocate((8, BLTREE_SIZE), 'u4', target=ol.bank0)

ltree_codes = pynq.allocate((8, LTREE_SIZE), 'u4', target=ol.bank0)
dtree_codes = pynq.allocate((8, DTREE_SIZE), 'u4', target=ol.bank0)
bltree_codes = pynq.allocate((8, BLTREE_SIZE), 'u4', target=ol.bank0)

ltree_blen = pynq.allocate((8, LTREE_SIZE), 'u4', target=ol.bank0)
dtree_blen = pynq.allocate((8, DTREE_SIZE), 'u4', target=ol.bank0)
bltree_blen = pynq.allocate((8, BLTREE_SIZE), 'u4', target=ol.bank0)

max_codes = pynq.allocate((8, MAXCODE_SIZE), 'u4', target=ol.bank0)

All of the new arrays are outputs of the tree generator and all of the input data is already on the card so we don't need to perform any syncing prior to calling the kernel.

In [13]:
treegen.call(ltree_freq, dtree_freq, bltree_freq,
             ltree_codes, dtree_codes, bltree_codes,
             ltree_blen, dtree_blen, bltree_blen,
             max_codes, 1024, 8 * BLOCK_SIZE, 8)

## Huffman Coding

The final stage is creating the huffman-coded data. Same as the previous kernels the first step is to look at the `args` dictionary to see what buffers we need and how to call the function

In [14]:
huffman.args

{'in_r': XrtArgument(name='in_r', index=1, type='ap_uint<512>*', mem='bank0'),
 'out_r': XrtArgument(name='out_r', index=2, type='ap_uint<512>*', mem='bank0'),
 'in_block_size': XrtArgument(name='in_block_size', index=3, type='unsigned int*', mem='bank0'),
 'compressd_size': XrtArgument(name='compressd_size', index=4, type='unsigned int*', mem='bank0'),
 'dyn_litmtree_codes': XrtArgument(name='dyn_litmtree_codes', index=5, type='unsigned int*', mem='bank0'),
 'dyn_distree_codes': XrtArgument(name='dyn_distree_codes', index=6, type='unsigned int*', mem='bank0'),
 'dyn_bitlentree_codes': XrtArgument(name='dyn_bitlentree_codes', index=7, type='unsigned int*', mem='bank0'),
 'dyn_litmtree_blen': XrtArgument(name='dyn_litmtree_blen', index=8, type='unsigned int*', mem='bank0'),
 'dyn_dtree_blen': XrtArgument(name='dyn_dtree_blen', index=9, type='unsigned int*', mem='bank0'),
 'dyn_bitlentree_blen': XrtArgument(name='dyn_bitlentree_blen', index=10, type='unsigned int*', mem='bank0'),
 'dyn_m

The only new buffer we need is the final output array. This is expected to be size of the original input array.

In [15]:
out_buffers = pynq.allocate((8,BLOCK_SIZE), 'u1', target=ol.bank0)

The coder can be called with the LZ77 buffer and the constructed tree to get the final compressed data.

In [16]:
huffman.call(lz77_buffers, out_buffers,
             compressed_size, uncompressed_size,
             ltree_codes, dtree_codes, bltree_codes,
             ltree_blen, dtree_blen, bltree_blen,
             max_codes, 1024, 8*BLOCK_SIZE)

The `compressed_size` array has now been updated with the new sizes of the output block and we can check that to ensure that everything worked correctly.

In [17]:
compressed_size.sync_from_device()
compressed_size

PynqBuffer([ 83335, 112135, 355840, 321234, 666936, 558682, 570153,
            508385], dtype=uint32)

## Creating the ZLIB File

We now have a set of DEFLATE encoded blocks but we need to package them up into a zlib data stream before we can decompress them with the built-in Python zlib library. A zlib file consists of:

 1. A 2-byte header identifying the file type
 2. A list of DEFLATE blocks ending with an empty block
 3. An Adler-32 checksum of the uncompressed data

The header and empty block are constants we can just define for this purpose and the `zlib` module provides a standalone `adler32` function we can use to calculate the checksum. More details of this structure can be found in [RFC1950](https://tools.ietf.org/html/rfc1950).

In [18]:
ZLIB_HEADER = b'\x78\x9c'
ZLIB_EMPTY_BLOCK = b'\x01\x00\x00\xff\xff'

import zlib
import struct
checksum = struct.pack('>I', zlib.adler32(test_data))

Similar to the LZ4 code shown in the previous notebooks we can use a `BytesIO` object to assemble the parts of the ZLIB stream.

In [19]:
import io
compressed_stream = io.BytesIO()
compressed_stream.write(ZLIB_HEADER)

for i in range(8):
    size = compressed_size[i]
    subbuf = out_buffers[i][0:size]
    subbuf.sync_from_device()
    compressed_stream.write(subbuf)
    
compressed_stream.write(ZLIB_EMPTY_BLOCK)
compressed_stream.write(checksum)

compressed_stream.seek(0)
compressed = compressed_stream.read()

And use the `zlib.decompress` function to check our compressed data stream for validity .

In [20]:
uncompressed = zlib.decompress(compressed)
uncompressed == test_data

True

We can compare the lengths of the compress and uncompressed streams to check the amount of compression. For the start of the bitstream we get about 2.6:1

In [21]:
len(uncompressed) / len(compressed)

2.6406582153680334

which compares favorably with the default setting for software zlib

In [22]:
len(uncompressed) / len(zlib.compress(test_data))

2.644351654365291

## Performance

To get a rough idea of the speed of the compressor we can run `%%timeit` on a cell containing all of the compute from the rest of the notebook. Note that this is a very crude measure as there is no overlapping of compute and communication or even compute of the three kernels.

In [23]:
%%timeit
in_buffers.sync_to_device()
uncompressed_size.sync_to_device()

wh1 = lz77.start(in_buffers, lz77_buffers,
                 compressed_size, uncompressed_size,
                 ltree_freq, dtree_freq,
                 1024, 8*BLOCK_SIZE)

wh2 = treegen.start(ltree_freq, dtree_freq, bltree_freq,
                    ltree_codes, dtree_codes, bltree_codes,
                    ltree_blen, dtree_blen, bltree_blen,
                    max_codes, 1024, 8 * BLOCK_SIZE, 8, waitfor=(wh1,))

wh3 = huffman.start(lz77_buffers, out_buffers,
                    compressed_size, uncompressed_size,
                    ltree_codes, dtree_codes, bltree_codes,
                    ltree_blen, dtree_blen, bltree_blen,
                    max_codes, 1024, 8*BLOCK_SIZE, waitfor=(wh2,))

zlib.adler32(test_data)
wh3.wait()
compressed_size.sync_from_device()

compressed_stream = io.BytesIO()
compressed_stream.write(ZLIB_HEADER)

for i in range(8):
    size = compressed_size[i]
    subbuf = out_buffers[i][0:size]
    subbuf.sync_from_device()
    compressed_stream.write(subbuf)
    
compressed_stream.write(ZLIB_EMPTY_BLOCK)
compressed_stream.write(checksum)

47.7 ms ± 9.28 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Even account for this we still get a reasonable speed-up over the built-in Python `zlib` module

In [24]:
%%timeit
zlib.compress(test_data)

510 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


For more benchmarking and performance data see the [documentation of the data compression library](https://xilinx.github.io/Vitis_Libraries/data_compression/)

### Cleaning up

Remember to *shutdown* this notebook at this point to ensure that all of the resources used are freed.

Copyright (C) 2020 Xilinx, Inc